# Pointer Generator Model

In [1]:
import torch
from PICOHelper import get_pico_datasets
from NewsroomHelper import get_newsroom_datasets
from models import Summarizer
from model_helpers import loss_function, error_function
from utils import get_index_words, produce_attention_visualization_file, summarize, get_text_triplets, produce_summary_files, run_rouge
from pytorch_helper import VariableBatchDataLoader, ModelManipulator, plot_learning_curves

## Parameters

In [ ]:
# training parameters
BATCH_SIZE = 64
NUM_EPOCHS = 1
LEARNING_RATE = 1e-2
# INITIAL_ACCUMULATOR_VALUE = 0.1
GAMMA = 1
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
BEAM_SIZE = 4

## Get Data

In [ ]:
# pico_dataset_train, pico_dataset_dev, pico_dataset_test = get_pico_datasets()
newsroom_dataset_train, newsroom_dataset_dev, newsroom_dataset_test, preprocessor = get_newsroom_datasets(with_oov=True)
word_vectors = preprocessor.word_vectors
start_index = preprocessor.word_indices['<start>']
end_index = preprocessor.word_indices['<end>']

## Create Model

In [ ]:
pointer_generator_model = Summarizer(preprocessor, start_index, end_index, num_hidden1=None, num_hidden2=None, with_coverage=True, gamma=GAMMA, with_pointer=True)

## Train and Save Model

In [ ]:
dataloader = VariableBatchDataLoader(newsroom_dataset_train, batch_size=BATCH_SIZE, shuffle=True)
optimizer = torch.optim.Adam(pointer_generator_model.parameters(),
                             lr=LEARNING_RATE)
# optimizer = torch.optim.Adagrad((pointer_generator_model.cuda() if USE_CUDA else pointer_generator_model).parameters(),
#                                 lr=LEARNING_RATE, initial_accumulator_value=INITIAL_ACCUMULATOR_VALUE)
model_manip = ModelManipulator(pointer_generator_model, optimizer, loss_function, error_function, use_cuda=USE_CUDA)
train_stats, val_stats = model_manip.train(dataloader, NUM_EPOCHS, dataset_val=newsroom_dataset_dev, stats_every=10, verbose_every=10)

In [ ]:
torch.save(pointer_generator_model, 'models/pointer_generator_temp.model')

In [ ]:
pointer_generator_model = torch.load('models/pointer_generator_temp.model')

## Plot

In [ ]:
plot_learning_curves(training_values=train_stats, validation_values=val_stats, figure_name='graphs/pointer_generator_training_temp')

In [ ]:
batch = newsroom_dataset_dev[0:5]
results = summarize(batch, pointer_generator_model, beam_size=BEAM_SIZE)

In [ ]:
summary_info = results[0]
triplets = get_text_triplets(batch, summary_info, preprocessor, pointer_gen=True)
for i,(text, reference_summary, decoded_summary) in enumerate(triplets):
    loss = summary_info[2][i]
    print("text", text)
    print("reference summary", reference_summary)
    print("decoded summary", decoded_summary)
    print(loss)

In [ ]:
summary_info = results[0]
i = 2

triplets = get_text_triplets(batch, summary_info, preprocessor)

text, reference_summary, decoded_summary = triplets[i]
attentions, p_gens = [[float(f) for f in vector[1:-1]] for vector in summary_info[4][i][:-1]], [float(f) for f in summary_info[5][i][:-1]]

produce_attention_visualization_file('graphs/attn_vis_data.json', text, " ".join(reference_summary), decoded_summary, attentions, p_gens)

In [ ]:
dataloader = VariableBatchDataLoader(newsroom_dataset_dev, batch_size=5, shuffle=True)
produce_summary_files(dataloader, pointer_generator_model, "rouge", beam_size=BEAM_SIZE, max_num_batch=10)

In [1]:
run_rouge()

NameError: name 'run_rouge' is not defined